In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.io import loadmat

In [ ]:
# @title Figure settings
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

In [ ]:
# @title Plotting Functions

def plot_glm_matrices(X, y, nt=50):
  """Show X and Y as heatmaps.

  Args:
    X (2D array): Design matrix.
    y (1D or 2D array): Target vector.

  """
  from matplotlib.colors import BoundaryNorm
  from mpl_toolkits.axes_grid1 import make_axes_locatable
  Y = np.c_[y]  # Ensure Y is 2D and skinny

  f, (ax_x, ax_y) = plt.subplots(
    ncols=2,
    figsize=(6, 8),
    sharey=True,
    gridspec_kw=dict(width_ratios=(5, 1)),
  )
  norm = BoundaryNorm([-1, -.2, .2, 1], 256)
  imx = ax_x.pcolormesh(X[:nt], cmap="coolwarm", norm=norm)

  ax_x.set(
    title="X\n(lagged stimulus)",
    xlabel="Time lag (time bins)",
    xticks=[4, 14, 24],
    xticklabels=['-20', '-10', '0'],
    ylabel="Time point (time bins)",
  )
  plt.setp(ax_x.spines.values(), visible=True)

  divx = make_axes_locatable(ax_x)
  caxx = divx.append_axes("right", size="5%", pad=0.1)
  cbarx = f.colorbar(imx, cax=caxx)
  cbarx.set_ticks([-.6, 0, .6])
  cbarx.set_ticklabels(np.sort(np.unique(X)))

  norm = BoundaryNorm(np.arange(y.max() + 1), 256)
  imy = ax_y.pcolormesh(Y[:nt], cmap="Blues", norm=norm)
  ax_y.set(
    title="y\n(spike count)",
    xticks=[]
  )
  ax_y.invert_yaxis()
  plt.setp(ax_y.spines.values(), visible=True)

  divy = make_axes_locatable(ax_y)
  caxy = divy.append_axes("right", size="30%", pad=0.1)
  cbary = f.colorbar(imy, cax=caxy)
  cbary.set_ticks(np.arange(y.max()) + .5)
  cbary.set_ticklabels(np.arange(y.max()))

# we will call this for comparison of two filters
def plot_spike_filter(theta, dt, show=True, **kws):
  """Plot estimated weights based on time lag model.

  Args:
    theta (1D array): Filter weights, not including DC term.
    dt (number): Duration of each time bin.
    kws: Pass additional keyword arguments to plot()
    show (boolean): To plt.show or not the plot.
  """
  d = len(theta)
  t = np.arange(-d + 1, 1) * dt

  ax = plt.gca()
  ax.plot(t, theta, marker="o", **kws)
  ax.axhline(0, color=".2", linestyle="--", zorder=1)
  ax.set(
    xlabel="Time before spike (s)",
    ylabel="Filter weight",
  )
  if show:
    plt.show()


In [ ]:
# @title Data retrieval and loading
import os
import hashlib
import requests

fname = "RGCdata.mat"
url = "https://osf.io/mzujs/download"
expected_md5 = "1b2977453020bce5319f2608c94d38d0"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    elif hashlib.md5(r.content).hexdigest() != expected_md5:
      print("!!! Data download appears corrupted !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

***Acknowledgement***:

This notebook is modified from a [introduction to GLM tutoral](https://github.com/NeuromatchAcademy/course-content) created by Neuromatch Academy.

---

# **P2**  GLMs for Encoding

#### The objective is to model a retinal ganglion cell spike train by fitting a temporal receptive field. First with a Linear-Gaussian GLM (also known as ordinary least-squares regression model) and then with a Poisson GLM (aka "Linear-Nonlinear-Poisson" model).

This tutorial is designed to run with retinal ganglion cell spike train data from [Uzzell & Chichilnisky 2004](https://journals.physiology.org/doi/full/10.1152/jn.01171.2003?url_ver=Z39.88-2003&rfr_id=ori:rid:crossref.org&rfr_dat=cr_pub%20%200pubmed).


we use data from an experiment that presented a screen which randomly alternated between two luminance values and recorded responses from retinal ganglion cell (RGC), a type of neuron in the retina in the back of the eye. This kind of visual stimulus is called a "full-field flicker", and it was presented at ~120Hz (ie. the stimulus presented on the screen was refreshed about every 8ms). These same time bins were used to count the number of spikes emitted by each neuron.

The file `RGCdata.mat` contains three variables:

- `Stim`, the stimulus intensity at each time point. It is an array with shape $T \times 1$, where $T=144051$.

-  `SpCounts`, the binned spike counts for 2 ON cells, and 2 OFF cells. It is a $144051 \times 4$ array, and each column has counts for a different cell.

- `dtStim`, the size of a single time bin (in seconds), which is needed for computing model output in units of spikes / s. The stimulus frame rate is given by `1 / dtStim`.

In [ ]:
# Read the data from a Matlab file

data = loadmat('RGCdata.mat')  # loadmat is a function in scipy.io
dt_stim = data['dtStim'].item()  # .item extracts a scalar value

# Extract the stimulus intensity
stim = data['Stim'].squeeze()  # .squeeze removes dimensions with 1 element

# Extract the spike counts for one cell
cellnum = 2
spikes = data['SpCounts'][:, cellnum]

# Choose not to use all of the timepoints in the dataset, for speed
keep_timepoints = 20000
stim = stim[:keep_timepoints]
spikes = spikes[:keep_timepoints]

First, let us visualize the changes in stimulus intensities and spike counts over time.



In [ ]:
# Total number of points to plot
nt = 120
timepoints = np.arange(nt)
time = timepoints * dt_stim

f, (ax_stim, ax_spikes) = plt.subplots(nrows=2, sharex=True, figsize=(8, 5))
ax_stim.plot(time, stim[timepoints])
ax_stim.set_ylabel('Stimulus intensity')

# Set x-axis at y=0 for ax_stim
ax_stim.spines['bottom'].set_position(('data', 0))
ax_stim.spines['top'].set_visible(False)
ax_stim.spines['right'].set_visible(False)
ax_stim.xaxis.set_ticks_position('bottom')
ax_stim.set_axisbelow(False)

ax_spikes.plot(time, spikes[timepoints])
ax_spikes.set_xlabel('Time (s)')
ax_spikes.set_ylabel('Number of spikes')

f.tight_layout()

### Create design matrix

Our goal is to predict the cell's activity from the stimulus intensities preceding it. That will help us understand how RGCs process information over time. To do so, we first need to create the *design matrix* for this model, which organizes the stimulus intensities in matrix form such that the $i$th row has the stimulus frames preceding timepoint $i$.

We will create the design matrix $\mathbf{X}$ using $d=25$ time lags. That is, $\mathbf{X}$ should be a $T \times d$ matrix. $d = 25$ (about 200 ms) is a choice we're making based on our prior knowledge of the temporal window that influences RGC responses. In practice, you might not know the right duration to use.

In [ ]:
d = 25
# Construct a matrix where each row has the d frames of
# the stimulus preceding and including timepoint t
T = len(stim)
# Total number of timepoints (hint: number of stimulus frames)
X = np.zeros((T, d))
# Add placeholder zeros to the earlest sliding windows
padded_stim = np.concatenate([np.zeros(d - 1), stim])

for t in range(T):
  X[t] = padded_stim[t:t+25]

# Visualize
plot_glm_matrices(X, spikes, nt=50)

### Fit Linear-Gaussian regression model


First, we will use the design matrix to compute the maximum likelihood estimate for a linear-Gaussian GLM (aka "general linear model"), simply assuming the firing rate (0 or 1) is a linear sum of past stimuli weighed by the filter $\theta$, for which we will infer the value for.

There are two approaches to obtain $\theta$, and interestingly for the simple linear filter, both approaches achieve the same analytical solution.

#### **Approach 1**: Mean Square Error (MSE)

We can think about the temporal receptive field as a function, and the best parameters are obtained when the average error between the function prediction $\hat y$ and the true $y$ is the smallest.

\begin{equation}
\boldsymbol{\theta}^*=\underset{\boldsymbol{\theta}}{\operatorname{argmin}}\|X \boldsymbol{\theta}-\boldsymbol{y}\|_2^2=\sum_{t=1}^T\left(y_t-\boldsymbol{\theta}^{\top} \boldsymbol{x}_t\right)^2
\end{equation}

Since this objective function is convex, the minimum is achieved when the gradient is zero:

\begin{equation}
\begin{aligned}
\frac{\partial\|X \boldsymbol{\theta}-\boldsymbol{y}\|_2^2}{\partial \boldsymbol{\theta}} & =2 X^{\top}(X \boldsymbol{\theta}-\boldsymbol{y})=0 \\
\Rightarrow \boldsymbol{\theta}_{\mathrm{MSE}} & =\left(X^{\top} X\right)^{-1} X^{\top} \boldsymbol{y}
\end{aligned}
\end{equation}

#### **Approach 2**: Maximum Likelihood Estimator (MLE)
We can think about the temporal receptive field as a generative model, then we would want a model with parameters that maximize the likelihood of seeing the observed data (the firing rate). Given gaussian noise,  

\begin{equation}
\boldsymbol{\theta}^*=\underset{\boldsymbol{\theta}}{\operatorname{argmax}} \log L (\boldsymbol{\theta} \mid X, \boldsymbol{y})=-\frac{Td}{2} \log 2 \pi \sigma^2-\frac{1}{2 \sigma^2}(\boldsymbol{y}-X \boldsymbol{\theta})^{\top}(\boldsymbol{y}-X \boldsymbol{\theta})
\end{equation}

The objective is quadratic in $\theta$, meaning that we can obtain the maximum when gradient is zero:

\begin{equation}
\begin{aligned}
& \frac{\partial \log L (\boldsymbol{\theta} \mid X, \boldsymbol{y})}{\partial \boldsymbol{\theta}}=-\frac{1}{\sigma^2} X^{\top}(X \boldsymbol{\theta}-\boldsymbol{y})=0 \\
& \Rightarrow \boldsymbol{\theta}_{\mathrm{MLE}}=\left(X^{\top} X\right)^{-1} X^{\top} \boldsymbol{y}
\end{aligned}
\end{equation}

We obtain the same close-form solution!


Note: Before we can apply this equation, we would need to augment the design matrix to account for the mean of $y$, because the spike counts are all $\geq 0$. We do this by adding a constant column of 1's to the design matrix, which will allow the model to learn an additive offset weight.

In [ ]:
# Build the full design matrix with the constant column
y = spikes
constant = np.ones_like(y)
X = np.column_stack([constant,X])

# Get the MLE weights for the LG model
theta = np.linalg.inv(X.T @ X) @ X.T @ y
theta_lg = theta[1:]

Plot the resulting maximum likelihood filter estimate:

In [ ]:
# plot temporal filter we have inferred:

d = len(theta_lg) # length filtered weights
t = np.arange(-d + 1, 1) * dt_stim # actual time in seconds

ax = plt.gca()
ax.plot(t, theta_lg, marker="o")
ax.axhline(0, color=".2", linestyle="--", zorder=1)
ax.set(
  xlabel="Time before spike (s)",
  ylabel="Filter weight",
)
plt.show()

Now we can also visualize how well our temporal filter model describe this neuron's firing patter by using it to predicting the spiking. The way to do it is just to compute predicted  $\mathbf{\hat y} = X \boldsymbol{\hat \theta}$, and plot it agaist $\mathbf{y}$ firing:

In [ ]:
yhat = X @ theta

# Visualize
nt = 50
t0 = 120

t = np.arange(t0, t0 + nt) * dt_stim

f, ax = plt.subplots()
lines = ax.stem(t, spikes[:nt], use_line_collection=True)
plt.setp(lines, color=".5")
lines[-1].set_zorder(1)
ax.plot(t, yhat[:nt], linewidth = 3)
ax.set(
    xlabel="Time (s)",
    ylabel="Spikes",
)
ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
ax.legend([lines[0], yhat], ["Spikes", "Predicted"])

plt.show()


* Is this a good model?

The prediction line more-or-less follows the bumps in the spikes, but it never predicts as many spikes as are actually observed. And, more troublingly, it's predicting *negative* spikes for some time points.

The Poisson GLM will help to address these failures.

<br/>

---

### **Bonus:**Linear-Nonlinear-Poisson GLM


In this exercise, we will use [`scipy.optimize.minimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) to compute maximum likelihood estimates for the filter weights in the Poisson GLM model with an exponential nonlinearity (LNP: Linear-Nonlinear-Poisson).

In practice, this will involve filling out two functions.

- The first should be an *objective function* that takes a design matrix, a spike count vector, and a vector of parameters. It should return a negative log likelihood.
- The second function should take `stim` and `spikes`, build the design matrix and then use `minimize` internally, and return the MLE parameters.

What should the objective function look like? We want it to return the negative log likelihood: $-\log P(y \mid \mathbf{X}, \theta).$

In the Poisson GLM,

\begin{equation}
\log P(\mathbf{y} \mid \mathbf{X}, \theta) = \sum_t \log P(y_t \mid \mathbf{x_t},\theta),
\end{equation}

where

\begin{equation}
P(y_t \mid \mathbf{x_t}, \theta) = \frac{\lambda_t^{y_t}\exp(-\lambda_t)}{y_t!} \text{, with rate } \lambda_t = \exp(\mathbf{x_t}^{\top} \theta).
\end{equation}

Now, taking the log likelihood for all the data we obtain:

\begin{equation}
\log P(\mathbf{y} \mid X, \theta) = \sum_t( y_t \log\left(\lambda_t) - \lambda_t - \log(y_t !)\right).
\end{equation}

Because we are going to minimize the negative log likelihood with respect to the parameters $\theta$, we can ignore the last term that does not depend on $\theta$. For faster implementation, let us rewrite this in matrix notation:

\begin{equation}
\mathbf{y}^{\top} \log(\mathbf{\lambda}) - \mathbf{1}^{\top} \mathbf{\lambda} \text{, with  rate } \mathbf{\lambda} = \exp(\mathbf{X} \theta)
\end{equation}


In [ ]:
def neg_log_lik_lnp(theta, X, y):
  """Return -loglike for the Poisson GLM model.

  Args:
    theta (1D array): Parameter vector.
    X (2D array): Full design matrix.
    y (1D array): Data values.

  Returns:
    number: Negative log likelihood.

  """
  # Compute the Poisson log likelihood
  rate = np.exp(X @ theta)
  log_lik = y @ np.log(rate) - rate.sum()
  return -log_lik


def fit_lnp(stim, spikes, d=25):
  """Obtain MLE parameters for the Poisson GLM.

  Args:
    stim (1D array): Stimulus values at each timepoint
    spikes (1D array): Spike counts measured at each timepoint
    d (number): Number of time lags to use.

  Returns:
    1D array: MLE parameters

  """

  # Build the design matrix
  y = spikes
  constant = np.ones_like(y)

  # Use a random vector of weights to start (mean 0, sd .2)
  x0 = np.random.normal(0, .2, d + 1)

  # Find parameters that minmize the negative log likelihood function
  res = minimize(neg_log_lik_lnp, x0, args=(X, y))

  return res["x"]


In [ ]:
# Fit LNP model
theta_lnp = fit_lnp(stim, spikes)

plot_spike_filter(theta_lg[1:], dt_stim, show=False, color=".5", label="LG")
plot_spike_filter(theta_lnp[1:], dt_stim, show=False, label="LNP")
plt.legend(loc="upper left")
plt.show()

Plotting the LG and LNP weights together, we see that they are broadly similar, but the LNP weights are generally larger. What does that mean for the model's ability to *predict* spikes? It will turn out that LNP model does a better job of fitting the actual spiking data. Importantly, it never predicts negative spikes!

If you are interested, try to plot the predicted $\mathbf {\hat y} $ using the filter inferred from the LNP model.